In [1]:
import os
import gluonnlp as nlp
import tensorflow as tf
import numpy as np
from gluonnlp.data import SentencepieceTokenizer # Use the tokenizer that SKT used to train KoGPT2
from transformers import TFGPT2LMHeadModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-06-29 05:08:02.649204: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-29 05:08:04.888719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 05:08:04.890375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 05:08:04.890953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 05:08:04.891790: I tensorflow/core/

In [2]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
    
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [3]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = f'{BASE_MODEL_PATH}/gpt2_kor_tokenizer.spiece'

In [5]:
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(
    TOKENIZER_PATH,
    mask_token=None,
    sep_token=None,
    cls_token=None,
    unknown_token='<unk>',
    padding_token='<pad>',
    bos_token='<s>',
    eos_token='</s>'
)

In [6]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value
    
    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)
        
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

In [7]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]
        outputs = model(input_ids)[:, -1, :]
        
        if greedy:
            # Select the most probable word and convert it to text
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            # Select word randomly based on probability distribution and convert it to text
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        
        # Stop when eos token generated
        if gen == vocab.eos_token:
            break
        
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)
    
    return sent

In [8]:
generate_sent('오늘', gpt_model, greedy=True)

'오늘은 그녀와 함께                                                                                               '

In [9]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때 가서 보내 주었다 니 무시나 내가  대방청해서 하께'

In [10]:
DATA_IN_PATH = './dataset/jjaltoon_scripts_raw/'
TRAIN_DATA_FILE = '1.txt'

In [11]:
sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

In [12]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

In [13]:
input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [14]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [15]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [16]:
def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    
    pred *= mask
    acc = train_accuracy(real, pred)
    
    return tf.reduce_mean(acc)

In [17]:
gpt_model.compile(loss=loss_function,
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=[accuracy_function])

In [18]:
history = gpt_model.fit(input_data, output_data,
                        batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                        validation_split=0.1)

Epoch 1/10
6/6 [==============================] - 18s 596ms/step - loss: 4.9763 - accuracy_function: 0.1510 - val_loss: 3.9778 - val_accuracy_function: 0.1806
Epoch 2/10
6/6 [==============================] - 1s 156ms/step - loss: 3.8142 - accuracy_function: 0.1997 - val_loss: 3.6179 - val_accuracy_function: 0.2141
Epoch 3/10
6/6 [==============================] - 1s 157ms/step - loss: 3.3296 - accuracy_function: 0.2255 - val_loss: 3.3531 - val_accuracy_function: 0.2385
Epoch 4/10
6/6 [==============================] - 1s 156ms/step - loss: 2.9421 - accuracy_function: 0.2524 - val_loss: 3.1796 - val_accuracy_function: 0.2610
Epoch 5/10
6/6 [==============================] - 1s 156ms/step - loss: 2.6417 - accuracy_function: 0.2728 - val_loss: 3.0478 - val_accuracy_function: 0.2811
Epoch 6/10
6/6 [==============================] - 1s 157ms/step - loss: 2.3545 - accuracy_function: 0.2933 - val_loss: 2.9264 - val_accuracy_function: 0.3014
Epoch 7/10
6/6 [==============================] - 1

In [19]:
DATA_OUT_PATH = './data/out'
model_name = 'tf2_gpt2_finetuned_model'

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)
loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data/out/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [22]:
generate_sent('이때', loaded_gpt_model, greedy=True)

'이때                                                                                                    '

In [23]:
generate_sent('이때', loaded_gpt_model, top_k=0, top_p=0.75)

'이때                         ,         선배님  ,                                          저,               '